# GTF GFF3 PARSER
########################################################################
## 04/08/2020
## By Xiang Li,
## lux@gwu.edu
## Peng's Lab1
## Ver.1.0
########################################################################


In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
def DIR_CHECK_CREATE(Input_Path):
    if (not os.path.isdir(Input_Path)):
        print ("New Dir Made is" + Input_Path)
        os.mkdir(Input_Path)
    else: 
        print ('Dir Exists')
def Add_common_header(df,common_header):
    name_list=[common_header]*len(df.columns)
    tuples = list(zip(name_list,df.columns))
    df.columns = pd.MultiIndex.from_tuples(tuples)
    return df

In [41]:
### gtf
def Get_Simple_Genes_gtf2bed(df, out_name):

    df_bed = []
    i=0
    ### Split last column, (gene_id, transcript_id and etc)
    df = pd.concat([df.drop([8],axis=1),df.iloc[:,8].str.split(pat=';',expand=True)], axis=1,ignore_index='True')
    ### Group by gene_id and transcript_id,
    ### If counting results are equal, means it is a simple gene.
    ### Some wierd loopole, some row does not have p_id (col 11), some has
    ### without p_id, then 10 is transcript_id
    df_no_p_id = df[df[12]=='']
    ### with p_id, then 11 is transcript_id
    df_with_p_id = df[df[12]!='']

    ### Iterating Each Gene Group and convert to bed format.
    for gene_id, gene_group in df_no_p_id.groupby([8]):
        if (len(gene_group[10].unique()) == 1):
            df_bed.append([gene_group.iloc[0,0], min(gene_group[3]), max(gene_group[4]), gene_id[9:-1], '.', gene_group.iloc[0,6]])
            i+=1
    for gene_id, gene_group in df_with_p_id.groupby([8]):
        if (len(gene_group[11].unique()) == 1):
            df_bed.append([gene_group.iloc[0,0], min(gene_group[3]), max(gene_group[4]), gene_id[9:-1], '.', gene_group.iloc[0,6]])
            i+=1
    ### Output        
    df_bed = pd.DataFrame(df_bed, columns=['#chr','start','end','name','score','strand']).sort_values(by=['#chr','start'])
    df_bed.to_csv(str(i)+'_'+out_name+'_Simple_Gene_list.bed', sep='\t', index=None)
    return df_bed

def Get_Genes_gtf2bed(_df, out_name):
    df_bed = []
    i=0
    ### Split last column, (gene_id, transcript_id and etc)
    df = pd.concat([_df.drop([8],axis=1),_df.iloc[:,8].str.split(pat=';',expand=True)], axis=1,ignore_index='True')
    ### Iterating Each Gene Group and convert to bed format.
    chr_list=list(['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15',
       'chr16', 'chr17', 'chr18', 'chr19', 'chr2', 'chr3', 'chr4', 'chr5',  'chr6', 'chr7',
       'chr8', 'chr9', 'chrX','chrY', 'chrM', 'chr20', 'chr21', 'chr22'])
    for chr_id, chr_group in df.groupby([0]):
        if(chr_id in chr_list):
            for gene_id, gene_group in chr_group.groupby([8]):
                if (gene_group.iloc[0,6]=='-'):
                    df_bed.append([gene_group.iloc[0,0], min(gene_group[3]), max(gene_group[4]), gene_id[9:-1], '.', gene_group.iloc[0,6], max(gene_group[4])])
                else:
                    df_bed.append([gene_group.iloc[0,0], min(gene_group[3]), max(gene_group[4]), gene_id[9:-1], '.', gene_group.iloc[0,6], min(gene_group[3])])
                i+=1
    ### Output        
    df_bed = pd.DataFrame(df_bed, columns=['#chr','start','end','name','score','strand', 'bin']).sort_values(by=['#chr','start'])
    df_bed_output = pd.DataFrame(data=df_bed.values, columns=['#chr','start','end','name','score','strand', 'bin']).sort_values(by=['#chr','start'])
    df_bed_output.loc[:,'start'] = df_bed_output.loc[:,'start'] -1  ## start -1 to make sure bed format start at 0
    df_bed_output.to_csv(str(i)+'_'+out_name+'_Gene_list.bed', sep='\t', index=None)
    return df_bed

## extened with respect to TSS
def extended_wrt_TSS_to_bed(_df_of_gtf, _Up_extension, _Down_extension, _out_name):
# df_of_gtf has to be the following bed6 format: 
# '#chr', 'start', 'end', 'name', 'score', 'strand'
    xx=pd.DataFrame(data=_df_of_gtf.values, columns=_df_of_gtf.columns)
    xx['end-'] = xx[xx['strand']=='-']['end']+_Up_extension
    xx['start-'] = xx[xx['strand']=='-']['end']-_Down_extension-1

    xx['start+'] = xx[xx['strand']=='+']['start']-_Up_extension-1
    xx['end+'] = xx[xx['strand']=='+']['start']+_Down_extension

    xx['start'] =  (xx['start-'].fillna(0) + xx['start+'].fillna(0)).apply(lambda x : int(x))
    xx['end'] = (xx['end-'].fillna(0) + xx['end+'].fillna(0)).apply(lambda x : int(x))

    xx.loc[:,['#chr','start','end','name','score','strand']].to_csv(str(len(xx))+_out_name+'_TSS_UP_'+str(_Up_extension)+
                                                                    '_Down_'+str(_Down_extension)+'.bed', sep='\t', index=None)
    return None

## extened with respect to TSS and genebody
def extended_wrt_TSS_TES_to_bed(_df_of_gtf, _Up_extension, _Down_extension, _out_name):
# df_of_gtf has to be the following bed6 format: 
# '#chr', 'start', 'end', 'name', 'score', 'strand'
    xx=pd.DataFrame(data=_df_of_gtf.values, columns=_df_of_gtf.columns)
# for strand -, TSS = end, TES = start
    xx['end-'] = xx[xx['strand']=='-']['end']+_Up_extension
    xx['start-'] = xx[xx['strand']=='-']['start']-_Down_extension-1

    # for strand +, TSS = start, TES = end
    xx['start+'] = xx[xx['strand']=='+']['start']-_Up_extension-1
    xx['end+'] = xx[xx['strand']=='+']['end']+_Down_extension

    xx['start'] = (xx['start-'].fillna(0) + xx['start+'].fillna(0)).apply(lambda x : int(x))
    xx['end'] = (xx['end-'].fillna(0) + xx['end+'].fillna(0)).apply(lambda x : int(x))

    xx.loc[:,['#chr','start','end','name','score','strand']].to_csv(str(len(xx))+_out_name+'_TSS_UP_'+str(_Up_extension)+'_TES_Down_'+str(_Down_extension)+'.bed', sep='\t', index=None)
    return None

#  Plus Extension

In [42]:
PATH_FOLDER=os.getcwd()
DIR_CHECK_CREATE(PATH_FOLDER)
INPUT_LIST=os.listdir(PATH_FOLDER)

NAME = '/mm10_2015.gtf'

df = pd.read_csv(PATH_FOLDER+NAME, sep='\t',skiprows=None, header=None, nrows=1)

df_bed = Get_Genes_gtf2bed(df, 'mm10_gene_Promoters')

Up_extension=50000
Down_extension=0

#extended_wrt_TSS_to_bed(df_bed, Up_extension, Down_extension,'mm10')


#xx = Get_Genes_gtf2bed(df_bed, 'mm10_genes')
#extended_wrt_TSS_TES_to_bed(xx, Up_extension, Down_extension,'_mm10')

Dir Exists


In [48]:
NAME = '/mm9_2015.gtf'

df = pd.read_csv(PATH_FOLDER+NAME, sep='\t',skiprows=None, header=None)

df_bed = Get_Genes_gtf2bed(df, 'mm9_genes')

Up_extension=50000
Down_extension=0

extended_wrt_TSS_to_bed(df_bed, Up_extension, Down_extension,'mm9')

#df_bed = Get_Genes_gtf2bed(df, 'mm9_genes')
extended_wrt_TSS_TES_to_bed(df_bed, Up_extension, Down_extension,'mm9')

In [4]:
## PARSING GTF file
PATH_FOLDER=os.getcwd()
DIR_CHECK_CREATE(PATH_FOLDER)
INPUT_LIST=os.listdir(PATH_FOLDER)

#NAME = '/mm9_Satellite.t'
NAME2= '/mm9_2015.gtf'

#df = pd.read_csv(PATH_FOLDER+NAME, sep='\t',skiprows=None, header=None ) #, nrows=200)
#df = pd.concat([df.drop([8],axis=1),df.iloc[:,8].str.split(pat=';',expand=True)], axis=1,ignore_index='True')

df = pd.read_csv(PATH_FOLDER+NAME2, sep='\t',skiprows=0, header=None)#, nrows=500)
#df2 = pd.concat([df.drop([8],axis=1),df.iloc[:,8].str.split(pat=';',expand=True)], axis=1,ignore_index='True')
df = Get_Genes_gtf2bed(df, 'mm9_genes_with_strand')
#df
#xx = Get_Genes_gtf2bed(df, 'mm9_genes')
#df = df.iloc[:,[5,6,7,11]]
#df.to_csv('mm9_Satellite.bed', sep='\t',index=None, header=None)

#df = pd.concat([df.drop([8],axis=1),df.iloc[:,8].str.split(pat=';',expand=True)], axis=1,ignore_index='True')
#df = df[df[2]=='three_UTR_region']
#df.iloc[:,[0,3,4,9,6]].drop_duplicates(subset=9).to_csv('hg38_3UTR.bed',sep='\t',header=None, index=None)

Dir Exists


,#chr,start,end,name,score,strand,bin
23665,chr1,3204562,3661579,Xkr4,.,-,3661579
18839,chr1,4280926,4399322,Rp1,.,-,4399322
20541,chr1,4481008,4487435,Sox17,.,-,4487435
14014,chr1,4763278,4775807,Mrpl15,.,-,4775807
12090,chr1,4797973,4836816,Lypla1,.,+,4797973
21395,chr1,4847774,4887990,Tcea1,.,+,4847774
18570,chr1,4899656,5060366,Rgs20,.,-,5060366
3095,chr1,5073253,5152630,Atp6v1h,.,+,5073253
16354,chr1,5578573,5596214,Oprk1,.,+,5578573
14861,chr1,5903787,5907479,Npbwr1,.,-,5907479
